In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from collections import OrderedDict

In [2]:
from ptranking.data.data_utils import LTRDataset, SPLIT_TYPE
data_id = 'MQ2007_Super'
#MQ2007内にあるもの
fold1 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S1.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
fold2 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S2.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
fold3 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S3.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
fold4 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S4.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
fold5 = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='S5.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)
test_data = LTRDataset(SPLIT_TYPE.Train, data_id=data_id, file='test.txt', batch_size=1, shuffle=True, presort=True, data_dict=None, eval_dict=None, buffer=False)

In [3]:
layers = [nn.Linear(46, 128),
          nn.ReLU(),
          nn.Linear(128, 64),
          nn.ReLU(),
          nn.Linear(64, 32),
          nn.ReLU(),
          nn.Linear(32, 1)]

In [4]:
class RankNet(nn.Module):
    def __init__(self, layers):
        super(RankNet, self).__init__()
        self.model = nn.Sequential(*layers)
        
    def forward(self, batch_ranking, label):
        batch_pred = self.model(batch_ranking) # [1, 40, 1]
        batch_sij = torch.squeeze(batch_pred, 0) - torch.squeeze(batch_pred, 2) # [40, 40]

        label_dim = torch.squeeze(label)
        label_diffs = torch.unsqueeze(label_dim, 1) - label # [40,40]
        batch_Sij = torch.clamp(label_diffs, -1, 1)
                
        # computes p_ij
        batch_pij = 1 / (1 + torch.exp(-batch_sij)) # [40, 40]
                
        # computes loss
        batch_loss = 0.5 * (1-batch_Sij) * batch_sij + torch.log(1+torch.exp(-batch_sij)) #[40, 40]
                
        # make upper triangular matrix
        batch_loss_triu = torch.triu(batch_loss, diagonal=1)

        # computes the mean value of a
        num_elements = (batch_loss_triu.size(1)*batch_loss_triu.size(1)-batch_loss_triu.size(1))/2
        batch_loss_mean = torch.sum(batch_loss)/num_elements
        
        #loss_list.append(batch_loss_mean)
                
        #print("batch_loss_mean", batch_loss_mean)
        
        return batch_loss_mean
    
    def predict(self, x):
        return self.model(x)

In [5]:
def DCG(sorted_labels, cutoff):
    denoms = torch.log2(torch.arange(2, cutoff+2))
    nums = torch.pow(2, sorted_labels[0:cutoff])-1
    dcg = sum(nums / denoms)
    return dcg

In [6]:
def nDCG(ideal, pred, k):
    dcg_f = DCG(pred, k)
    dcg = DCG(ideal, k)
    nDCG = dcg_f / dcg
    return nDCG

In [7]:
def compute_ndcg(rank,label,cutoff):     
    t = model.predict(rank)
    r = torch.argsort(t,dim=1,descending = True)
    ideal,_ = torch.sort(label,1,descending = True)
    ideal = torch.unsqueeze(ideal,2).reshape(-1)
    pred = torch.gather(label.unsqueeze(2),1,r).reshape(-1)
    ndcg = torch.nan_to_num(nDCG(ideal,pred,cutoff))
    return ndcg

In [8]:
def testing(data):
    with torch.no_grad():
        ndcg_sum = 0
        for _,batch_ranking,label in data:
        #computes ndcg
            ndcg = compute_ndcg(batch_ranking,label,5)
            ndcg_sum += ndcg  
    ndcg_sum /= len(data)
    return(ndcg_sum)

In [20]:
def training_loop(n_epochs,optimizer,model,data_list):
    count = 0
    best_ndcg = 0
    
    while count < len(data_list):
        # 最初の3つをtrainに
        train = data_list[0]+data_list[1]+data_list[2]
        # 1つをvalidationに
        val = data_list[3]
        # 最後をtestに
        test = data_list[4]
        
        # モデルの訓練
        for epoch in range(1, n_epochs + 1):
        
            # クエリごとの訓練
            batch_loss = 0
            batch_ndcg = 0
            for _,batch_ranking,label in train:
                batch_loss += model.forward(batch_ranking=batch_ranking, label=label)
                batch_ndcg += compute_ndcg(batch_ranking,label,5)
                    
            fold_loss_mean = batch_loss / (len(data_list[0])+len(data_list[1])+len(data_list[2]))
            fold_ndcg_mean = batch_ndcg / (len(data_list[0])+len(data_list[1])+len(data_list[2]))
            
            # パラメータ選択
            val_loss = 0
            val_ndcg = 0
            for _,batch_ranking,label in val:
                val_loss += model.forward(batch_ranking=batch_ranking, label=label)
                val_ndcg += compute_ndcg(batch_ranking,label,5)
                
            val_loss_mean = val_loss / len(val)
            val_ndcg_mean = val_ndcg / len(val)    
                
            if val_ndcg_mean > best_ndcg:
                best_ndcg = val_ndcg_mean
            
                # 訓練したモデルの保存
                torch.save(model.state_dict(), 'weight.pth')
            
            optimizer.zero_grad()
            fold_loss_mean.backward()
            optimizer.step()
            
        # パラメータの読み込み
        param = torch.load('weight.pth')
        model.load_state_dict(param)
        
        #test
        test_ndcg = testing(test)
        print("test", test_ndcg)
        
        #　右にひとつシフト ([1,2,3,4,5] -> [5,1,2,3,4]となる)
        data_list.insert(0,data_list.pop())
        
#         if epoch%10==0:
#             print("epoch: ", epoch, "loss: ", epoch_loss, "ndcg: ", epoch_ndcg)
        count += 1

In [13]:
data_list = [fold1,fold2,fold3,fold4,fold5]

In [14]:
# クラスのインスタンス化
model = RankNet(layers)

In [21]:
training_loop(
    n_epochs = 100,
    optimizer = torch.optim.SGD(model.parameters(),lr = 0.01),
    model = model,
    data_list = data_list,
    )

test tensor(0.1418)
test tensor(0.1203)
test tensor(0.1375)
test tensor(0.1554)
test tensor(0.2334)
